In [133]:
import json
import logging
import os
import re
from collections import Counter
from importlib import reload

import joblib
import numpy as np
import pandas as pd
import scipy
import tensorflow as tf
from sklearn.metrics import confusion_matrix


import pollock
from pollock.model.model import get_default_cell_classifier
# from pollock.preprocessing.preprocessing import get_expression_matrix



In [134]:
X_test = scipy.sparse.load_npz(
        '/Users/erikstorrs/Documents/ding/single-cell-classification/data/processed/X_test.npz')

In [135]:
metadata_df = pd.read_csv(
        '/Users/erikstorrs/Documents/ding/single-cell-classification/data/processed/test_metadata.tsv', sep='\t')
cell_encoder = joblib.load(
        '/Users/erikstorrs/Documents/ding/pollock/pollock/model/cell_type_encoder.pkl')

y_test = np.asarray(metadata_df['cell_type'])
# y_test_labels = np.copy(y_)
# y_test = cell_encoder.transform(y_test.reshape((-1, 1)))

In [137]:
list(Counter(y_test).keys())

['melanoma',
 'plasma',
 'macromononeutro',
 'malignantplasma',
 'tcell',
 'dc',
 'bcell',
 'ductal',
 'stroma',
 'fibroblast',
 'endothelial',
 'mast',
 'acinar']

In [5]:
X_test.shape, y_test.shape

((64064, 34241), (64064,))

In [6]:
reload(pollock)
reload(pollock.model.model)
# pollock.model.
# model = CellClassifier(weights_fp, train_mean, train_std, cell_encoder)
model = get_default_cell_classifier()

W0812 09:51:15.472088 4658537920 deprecation.py:323] From /Users/erikstorrs/miniconda3/envs/pollock/lib/python3.7/site-packages/tensorflow/python/keras/backend.py:4149: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [7]:
labels, probs = model.predict(X_test[:5000])

/Users/erikstorrs/Documents/ding/pollock/pollock/model/model.py:34: RuntimeWarning: divide by zero encountered in true_divide
  m = (m - mean) / std
/Users/erikstorrs/Documents/ding/pollock/pollock/model/model.py:34: RuntimeWarning: invalid value encountered in true_divide
  m = (m - mean) / std


In [ ]:
labels

In [ ]:
predictions = None
batch_size = 5000

for i in range(0, X_test.shape[0], batch_size):
    print(i)
    X = X_test[i:i + batch_size]
    X = normalize_matrix(X, mean=train_mean, std=train_std)
    
    if predictions is None:
        predictions = model.predict(X)
    else:
        predictions = np.concatenate((predictions, model.predict(X)), axis=0)

In [ ]:
np.max(predictions[:10], axis=1)

In [ ]:
labels = np.asarray([cell_encoder.categories_[0][i] for i in np.argmax(predictions, axis=1)])

In [ ]:
correct, wrong = [], []

for i, (ground_truth, predicted) in enumerate(zip(y_test, labels)):
    if ground_truth == predicted:
        correct.append(i)
    else:
        wrong.append(i)
        
len(correct), len(wrong), len(correct) / (len(correct) + len(wrong))

In [ ]:

Counter(labels[wrong]).most_common()

In [ ]:
total = Counter(y_test)
ground_truth_correct = Counter(y_test[correct])
ground_truth_wrong = Counter(y_test[wrong])

predicted_correct = Counter(labels[correct])
predicted_wrong = Counter(labels[wrong])

In [ ]:
for cell, count in total.most_common():
    print(cell)
    print(ground_truth_correct[cell] / count)
    print(ground_truth_wrong[cell] / count)
    print(predicted_wrong[cell] / count)

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test[:20000], labels)

cf_df = pd.DataFrame(data=cm, columns=cell_encoder.categories_, index=cell_encoder.categories_)

cf_df

In [ ]:
confident_idxs = np.asarray([i for i, conf in enumerate(np.max(predictions, axis=1))
                            if conf > .99])

print(len(confident_idxs))


cm = confusion_matrix(y_test[confident_idxs], labels[confident_idxs])

cf_df = pd.DataFrame(data=cm, columns=cell_encoder.categories_, index=cell_encoder.categories_)

cf_df

In [ ]:
import seaborn as sns

sns.heatmap(cf_df.values, annot=True)

## test out a test dataset

In [89]:
sample = '27522_1'

fp = f'/Users/erikstorrs/Documents/ding/single-cell-classification/data/myeloma/expression/MMY_raw_expression_in_sample_{sample}.txt'
valid_df = pd.read_csv('/Users/erikstorrs/Documents/ding/single-cell-classification/data/myeloma/metadata/barcode_cell_type_summary.txt',
                      sep='\t')
valid_df = valid_df[valid_df['sample_id'] == sample]
valid_df.index = list(valid_df['cell_barcode'])
valid_df.head()


,sample_id,cell_barcode,cell_type,technique
27522_1_AAACCTGAGATGTCGG,27522_1,27522_1_AAACCTGAGATGTCGG,Tcell,sc
27522_1_AAACCTGAGCGCTCCA,27522_1,27522_1_AAACCTGAGCGCTCCA,MalignantPlasma,sc
27522_1_AAACCTGAGCGGCTTC,27522_1,27522_1_AAACCTGAGCGGCTTC,MalignantPlasma,sc
27522_1_AAACCTGAGCTGATAA,27522_1,27522_1_AAACCTGAGCTGATAA,MacroMonoNeutro,sc
27522_1_AAACCTGCACAAGTAA,27522_1,27522_1_AAACCTGCACAAGTAA,MalignantPlasma,sc


In [80]:
reload(pollock.preprocessing.preprocessing)
expression_matrix, samples, genes = pollock.preprocessing.preprocessing.get_expression_matrix(fp)

/Users/erikstorrs/miniconda3/envs/pollock/lib/python3.7/site-packages/scipy/sparse/_index.py:103: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray_sparse(i, j, x)


In [81]:
expression_matrix.shape, len(samples), len(genes)

((4543, 34241), 4543, 34241)

In [101]:
reload(pollock)
reload(pollock.model.model)
model = get_default_cell_classifier()

In [128]:
labels, probs = model.predict(expression_matrix)

/Users/erikstorrs/Documents/ding/pollock/pollock/model/model.py:34: RuntimeWarning: divide by zero encountered in true_divide
  m = (m - mean) / std
/Users/erikstorrs/Documents/ding/pollock/pollock/model/model.py:34: RuntimeWarning: invalid value encountered in true_divide
  m = (m - mean) / std


[]


In [129]:
labels[:10]

array(['tcell', 'malignantplasma', 'malignantplasma', 'macromononeutro',
       'malignantplasma', 'tcell', 'tcell', 'malignantplasma', 'bcell',
       'malignantplasma'], dtype='<U32')

In [130]:
Counter(labels).most_common()

[('malignantplasma', 2490),
 ('macromononeutro', 1374),
 ('tcell', 365),
 ('bcell', 272),
 ('dc', 34),
 ('fibroblast', 5),
 ('melanoma', 2),
 ('plasma', 1)]

In [131]:
ground_truth = [valid_df.loc[s]['cell_type'].lower() for s in samples]

In [132]:
## score the cells
lbs = sorted(set(labels).union(set(ground_truth)))
cm = confusion_matrix(ground_truth, labels, labels=lbs)
cm_df = pd.DataFrame(data=cm, columns=lbs, index=lbs)

cm_df

,bcell,dc,fibroblast,macromononeutro,malignantplasma,melanoma,plasma,tcell
bcell,269,1,2,2,16,1,0,8
dc,0,32,0,17,0,0,0,0
fibroblast,0,0,0,0,0,0,0,0
macromononeutro,2,0,2,1325,24,0,1,0
malignantplasma,0,1,1,28,2447,0,0,0
melanoma,0,0,0,0,0,0,0,0
plasma,0,0,0,0,0,0,0,0
tcell,1,0,0,2,3,1,0,357


In [7]:

o = 1815

In [8]:
for i in range(1, 10):
    print(i * 10, (i / 10) * o)

10 181.5
20 363.0
30 544.5
40 726.0
50 907.5
60 1089.0
70 1270.5
80 1452.0
90 1633.5
